https://blog.datawrapper.de/weekly-ridgeline-plot/

In [1]:
import pandas as pd

from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
def get_dem_data(url):
    import requests
    from osgeo import gdal
    r = requests.get(url)
    vsipath = '/vsimem/dem'
    gdal.FileFromMemBuffer(vsipath, r.content)
    return gdal.Open(vsipath)

In [10]:
dataset = get_dem_data("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/ivindo_river.tif")
dataset_array = dataset.ReadAsArray()[::144, ::24]

print(dataset_array.shape)
dataset_array[:5, :5]

(26, 151)


array([[529, 484, 543, 538, 528],
       [529, 537, 538, 534, 491],
       [502, 522, 516, 490, 535],
       [601, 544, 542, 528, 537],
       [540, 532, 521, 528, 511]], dtype=int16)

In [11]:
df = pd.DataFrame.from_records([
    (j, i, a)
    for i, r in enumerate(dataset_array)
    for j, a in enumerate(r)
], columns=["x", "y", "h"])

m = 5
a = m / (df.h.max() - df.h.min())
b = -a * df.h.min()
df.h = a * df.h + b

print(df.shape)
df.head()

(3926, 3)


,x,y,h
0,0,0,3.575064
1,1,0,3.002545
2,2,0,3.753181
3,3,0,3.689567
4,4,0,3.562341


In [13]:
ggplot(df) + \
    geom_area_ridges(aes("x", "y", height="h"), \
                     stat='identity', \
                     size=1, fill="white", alpha=1,
                     sampling=sampling_pick(df.shape[0])) + \
    scale_y_continuous(trans='reverse') + \
    ggsize(800, 800)